# Planning permission

### Setup environment
Notebook uses OpenAI API. Use an existing key or [create a new key](https://platform.openai.com/account/api-keys)

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### Setup logger

In [2]:
import sys
import logging

# Load logging level from environment or set default level to ERROR
env_logging_level = os.environ.get("LOGGING_LEVEL", "ERROR")
level = getattr(logging, env_logging_level.upper(), logging.ERROR)

logging.basicConfig(stream=sys.stdout, level=level)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Import dependence 

In [3]:
from pathlib import Path
from IPython.display import Markdown
from llama_index import SimpleDirectoryReader, LLMPredictor, ServiceContext, KeywordTableIndex, download_loader, VectorStoreIndex
from langchain import OpenAI

INFO:numexpr.utils:Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
Note: NumExpr detected 10 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
NumExpr defaulting to 8 threads.


### Import Llama hub dependence

In [4]:
PDFMinerReader = download_loader("PDFMinerReader")
loader = PDFMinerReader()

### Setup LLM, vector store and context

In [5]:
# load index
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003"))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [6]:
documents = loader.load_data(file=Path(os.environ.get("PLANNING_PERMISSION_FILE_PATH")))
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()

In [7]:
response = query_engine.query("Behöver jag söka bygglov för en altan?")

In [8]:
display(Markdown(f"<b>{response}</b>"))

<b>
Ja, du behöver söka bygglov för en altan om den är högre än 1,8 meter, placeras mindre än 4,5 meter från gränsen, eller om den kan betraktas som en volymökning eller väsentligt ändrar fasaden.</b>